In [82]:
import pandas as pd
from scipy.spatial.distance import cosine

In [83]:
task = 6
model = 'dandelin-vilt-b32-mlm-mattn_score_0.8486'

In [84]:
gold_df = pd.read_json(f'task{task}_val_soft_gold.json')[['id', 'value']]
pred_df = pd.read_json(f'Task {task} Results/{model}_val_soft.json')[['id', 'value']]

In [85]:
merged = pd.merge(pred_df, gold_df, how='inner', on='id')
# merged = merged[merged['id'] < 200000].reset_index(drop=True)
merged = merged[merged['id'] >= 200000].reset_index(drop=True)
merged

,id,value_x,value_y
0,210005,{'IDEOLOGICAL-INEQUALITY': 1.20756377119946e-1...,"{'NO': 0.8333333333, 'IDEOLOGICAL-INEQUALITY':..."
1,210008,{'IDEOLOGICAL-INEQUALITY': 0.01315187662839800...,"{'OBJECTIFICATION': 0.5, 'IDEOLOGICAL-INEQUALI..."
2,210014,"{'IDEOLOGICAL-INEQUALITY': 0.000302606786135, ...","{'NO': 0.1666666667, 'IDEOLOGICAL-INEQUALITY':..."
3,210016,"{'IDEOLOGICAL-INEQUALITY': 0.9977498650550841,...","{'NO': 0.1666666667, 'MISOGYNY-NON-SEXUAL-VIOL..."
4,210023,"{'IDEOLOGICAL-INEQUALITY': 0.9448103308677671,...","{'STEREOTYPING-DOMINANCE': 0.5, 'IDEOLOGICAL-I..."
...,...,...,...
417,211982,{'IDEOLOGICAL-INEQUALITY': 4.766206873618728e-...,"{'NO': 1.0, 'IDEOLOGICAL-INEQUALITY': 0.0, 'ST..."
418,211993,{'IDEOLOGICAL-INEQUALITY': 0.07358453422784801...,"{'NO': 0.8333333333, 'STEREOTYPING-DOMINANCE':..."
419,211994,{'IDEOLOGICAL-INEQUALITY': 4.391968832351268e-...,"{'IDEOLOGICAL-INEQUALITY': 0.1666666667, 'NO':..."
420,211995,{'IDEOLOGICAL-INEQUALITY': 1.973896779361439e-...,"{'NO': 0.8333333333, 'IDEOLOGICAL-INEQUALITY':..."


In [86]:
def dict_to_vector(d, keys):
    return [d.get(key, 0) for key in keys]

In [87]:
keys = set()
for d in merged['value_x']:
    keys.update(d.keys())
for d in merged['value_y']:
    keys.update(d.keys())
keys = list(keys)

In [88]:
# Convert value_x and value_y to vectors
merged['vector_x'] = merged['value_x'].apply(lambda x: dict_to_vector(x, keys))
merged['vector_y'] = merged['value_y'].apply(lambda y: dict_to_vector(y, keys))

# Calculate cosine similarity
similarities = []
for i, row in merged.iterrows():
    similarity = 1 - cosine(row['vector_x'], row['vector_y'])  # Cosine similarity
    similarities.append(similarity)

merged['similarity'] = similarities

# Find the most similar and most dissimilar samples
most_similar = merged.iloc[merged['similarity'].idxmax()]
most_dissimilar = merged.iloc[merged['similarity'].idxmin()]

print("Most Similar Sample:")
print(most_similar)

print("\nMost Dissimilar Sample:")
print(most_dissimilar)

Most Similar Sample:
id                                                       210815
value_x       {'IDEOLOGICAL-INEQUALITY': 3.057666786432378e-...
value_y       {'NO': 1.0, 'IDEOLOGICAL-INEQUALITY': 0.0, 'ST...
vector_x      [3.057666786432378e-09, 1.3841078505549831e-08...
vector_y                         [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
similarity                                                  1.0
Name: 153, dtype: object

Most Dissimilar Sample:
id                                                       210355
value_x       {'IDEOLOGICAL-INEQUALITY': 1.5460884696949502e...
value_y       {'NO': 1.0, 'IDEOLOGICAL-INEQUALITY': 0.0, 'ST...
vector_x      [1.5460884696949502e-06, 0.053245760500431005,...
vector_y                         [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
similarity                                                  0.0
Name: 64, dtype: object
